# Basic Tutorial: BERT Models with *Giotto-Deep* and Topological Pruning

#### Author: Henry Kirveslahti

In this tutorial we demonstrate how to run a pre-trained BERT model with *Giotto-Deep*, as well as showcase some topological pruning methods. These pruning methods are based on the preprint *https://arxiv.org/pdf/2206.15195.pdf* by I.Perez & R.Reinauer.

We will consider the NLP problem of classifying (a subset of) sentences in the *Corpus of Linguistic Acceptability (CoLA)*. To this end, we deploy a pre-trained BERT model from *HuggingFace*. We will construct the attention graph from this model, which we will use to derive Persistent Images. We then create a model that takes these persistent images as an input, and based on this model, we can compute importance scores for each of the attention heads. By only using the attention heads with high score, we can create a pruned model.


The tutorial is organized as follows:

1. Deploying the HuggingFace Model

2. Fine-tuning the model using *Giotto-deep*

3. Retrieving the Topological Summaries

4. Training a topological model

5. Pruning


First we import some dependencies:


In [ ]:
import pandas as pd
import time
import datetime
import random
import wget
import os
import numpy as np
import torch
import torchtext
import copy
import sys
import math
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, SubsetRandomSampler
from transformers import (
                          BertTokenizer, 
                          BertForSequenceClassification,
                          AdamW,
                          BertConfig,
                          get_linear_schedule_with_warmup)
from torchtext.data.utils import get_tokenizer
from torch.optim import Adam, SparseAdam, SGD
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import ExponentialLR
from gdeep.models import FFNet
from gdeep.utility import DEVICE
from gdeep.trainer import Trainer
from gdeep.data import TransformingDataset
from gdeep.data.preprocessors import TokenizerTranslation
from gdeep.data.datasets import DatasetBuilder, FromArray, DataLoaderBuilder
from gdeep.models import ModelExtractor
from gdeep.analysis.interpretability import Interpreter
from gdeep.search import GiottoSummaryWriter
from gudhi.representations.vector_methods import PersistenceImage as gPI
from gudhi import RipsComplex as gRC

# Initialize the tensorboard writer

In order to analyse the results of your models, you need to start tensorboard. All data about the model, the training, the hyperparameters... will be stored there.

## How to start tensorboard
On the terminal, move inside the `/examples` folder. There run the following command:

```
tensorboard --logdir=runs
```

Then go [here](http://localhost:6006/) after the training step to visualise all the results.

In [ ]:
writer = GiottoSummaryWriter()

## 1. Deploying the BERT model

In this section we'll just download some data and deploy a BERT model from Hugging face. This lengthy part is quite technical and independent of *giotto-deep*, so we'll just need to click through this section to get to Section 2. **The only exception is the variable**
```
n_sentences_to_consider
```
**which you could set a bit higher to get more interesting results (at a computational expense).**

### 1.1 Preprocessing
Following the pre-print, we'll adapt the pre-processing steps from *https://github.com/MohamedAteya/BERT-Fine-Tuning-Sentence-Classification-for-CoLA/blob/master/BERT_Fine_Tuning_Sentence_Classification_for_CoLA.ipynb*

In [ ]:
n_sentences_to_consider=200
# Downloading the data
print('Downloading dataset...')

# The URL for the dataset zip file.
url = 'https://nyu-mll.github.io/CoLA/cola_public_1.1.zip'

# Download the file (if we haven't already)
if not os.path.exists('./cola_public_1.1.zip'):
    wget.download(url, './cola_public_1.1.zip')
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
if not os.path.exists('./cola_public/'):
    !unzip cola_public_1.1.zip
# Load the dataset into a pandas dataframe.
tmp_path=os.path.join('./cola_public','raw','in_domain_train.tsv')
df = pd.read_csv(tmp_path, delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])
# Get the lists of sentences and their labels.
sentences = df.sentence.values
labels = df.label.values

sentences=sentences[0:n_sentences_to_consider]
labels=labels[0:n_sentences_to_consider]
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

for sent in sentences:

    encoded_sent = tokenizer.encode( sent, add_special_tokens = True)
    
    # Add the encoded sentence to the list.
    input_ids.append(encoded_sent)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

print('Max length: ', max([len(sen) for sen in input_ids]))

Next we'll define a makeshift preprocessing step. This is a verbatim copy of the padding function from *tensorflow keras*. The point of the padding function is to add extra zeros to the input so that each of them is of the same size. We'll need this for the HuggingFace BERT model.

In [ ]:
MAX_LEN = 64
def pad_sequences(
    sequences,
    maxlen=None,
    dtype="int32",
    padding="pre",
    truncating="pre",
    value=0.0,
):
    if not hasattr(sequences, "__len__"):
        raise ValueError("`sequences` must be iterable.")
    num_samples = len(sequences)

    lengths = []
    sample_shape = ()
    flag = True
    for x in sequences:
        try:
            lengths.append(len(x))
            if flag and len(x):
                sample_shape = np.asarray(x).shape[1:]
                flag = False
        except TypeError as e:
            raise ValueError(
                "`sequences` must be a list of iterables. "
                f"Found non-iterable: {str(x)}"
            ) from e

    if maxlen is None:
        maxlen = np.max(lengths)

    is_dtype_str = np.issubdtype(dtype, np.str_) or np.issubdtype(
        dtype, np.unicode_
    )
    if isinstance(value, str) and dtype != object and not is_dtype_str:
        raise ValueError(
            f"`dtype` {dtype} is not compatible with `value`'s type: "
            f"{type(value)}\nYou should set `dtype=object` for variable length "
            "strings."
        )

    x = np.full((num_samples, maxlen) + sample_shape, value, dtype=dtype)
    for idx, s in enumerate(sequences):
        if not len(s):
            continue  # empty list/array was found
        if truncating == "pre":
            trunc = s[-maxlen:]
        elif truncating == "post":
            trunc = s[:maxlen]
        else:
            raise ValueError(f'Truncating type "{truncating}" not understood')

        # check `trunc` has expected shape
        trunc = np.asarray(trunc, dtype=dtype)
        if trunc.shape[1:] != sample_shape:
            raise ValueError(
                f"Shape of sample {trunc.shape[1:]} of sequence at "
                f"position {idx} is different from expected shape "
                f"{sample_shape}"
            )

        if padding == "post":
            x[idx, : len(trunc)] = trunc
        elif padding == "pre":
            x[idx, -len(trunc) :] = trunc
        else:
            raise ValueError(f'Padding type "{padding}" not understood')
    return x


input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")

In [ ]:
#Split the data to train and validation sets
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=13, test_size=0.1)

In [ ]:
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 2,
    output_attentions = True, 
    output_hidden_states = False
)

if(device.type=='cuda'):
    model.cuda()
if(device.type=='cpu'):
    model.cpu()

## 2. Fine-Tuning the Model with *Giotto-Deep*

To train a model with *giotto-deep* we'll need a) data, b) loss function, and optionally c) a performance metric. We define these below:

In [ ]:
dl_builder = DataLoaderBuilder((FromArray(train_inputs, train_labels), \
                                FromArray(validation_inputs, validation_labels)))
dl_tr, dl_val, _ = dl_builder.build(({"batch_size": 8}, {"batch_size": 8}))

loss = nn.CrossEntropyLoss()

As per the usual *giotto-deep* paradigm, we pass these to the Trainer.

In [ ]:
pipe = Trainer(model, (dl_tr, dl_val), loss, writer)

In [ ]:
# train the model with learning rate scheduler
pipe.train(
    Adam,
    4,
    False,
    lr_scheduler=ExponentialLR,
    scheduler_params={"gamma": 0.9},
    profiling=False,
    store_grad_layer_hist=True,
    writer_tag="line",
)

## 3. Retrieving the topological summaries

Now that we have trained our model and fine-tuned it, we would like to extract the topological summaries. For now we'll be interested in the attention matrices. We can retrieve them with the Model Extractor. Before diving deep into the attention matrices, let us first take a look at the inner workings of our HuggingFace model:

In [ ]:
ex = ModelExtractor(pipe.model, loss)

In [ ]:
# The Layers
layer_names = ex.get_layers_param().keys()
for tmp,layer in enumerate(layer_names):
    print(tmp,layer)

Next we will compute the attention masks. The point of the attention masks is that not all the sentences contain the same number of words, (nor tokenized words). The attention masks records the dummy binary variables indicating which of the elements of the padded inputs are actual tokens.

In [ ]:
# A helper function for computing attention masks
def attention_mask_from_input(input):
    mask=torch.zeros(input.shape)
    mask[torch.nonzero(input, as_tuple=True)]=1
    return(mask)

In [ ]:
# Next we'll look into the sizes of the different layers.
tmp,batch=next(enumerate(dl_tr))
b_input_ids = batch[0].to(device)
b_input_mask=attention_mask_from_input(b_input_ids)
b_labels = batch[1].to(device)
outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
layers = ex.get_activations(b_input_ids)
for ind_k,layer in enumerate(layers):
    print(ind_k,layer[0].shape)

From the above we see that the attention matrices are in the layers with indices 9,27,45,63,81,99,117,135,153,171,189,207. These corresponds to the Multi-head attention layers at different depths. Each of these contains 12 attention heads. Next we'll loop over the train data and store these layers, with the ultimate goal of constructing the Persistent images for each.

### 3.1. The Attention Matrices

We will get an attention matrix for each combination of the 144 (12x12) attention heads and $n$ sentences (that is, $144n$ matrices). 

For the persistence function, we are using the mean aggregation scheme, which means that the only data that we need is the mean of the attention scores of edges $e_{i,j}$ and $e_{j,i}$ (and of course the vertex birth times, which are all 0). These are obtained conveniently via matrix transpose. In the code snippet below this is achieved with

```
aggregated_attention=0.5*(raw_attention+raw_attention.transpose(0,1,3,2))
```

Note that in the stored attention matrices, the first index runs over $L$ (layers of the network) and the second over $H$ (heads in a fixed layer). The last two indices are, of course, for the tokens.


In [ ]:
# A helper function for computing attention matrices.
# In: a giotto-dataset returned by DataLoaderBuilder.build
# Out: 1) all_the_attention: A list of stacked attention matrices, one for each sample in the dataset
#         Each stack is of size (12,12,n_i,n_i),
#         where n_i is the length of the sentence (measured in number of tokens)
#         The attention matrices are already mean-aggregated
#      2) all_the_labels: The label associated with the sentences
#      3) raw_attentions: The raw attention matrices. Same shape as all_the_attention
#         Used only for plotting
def compute_attention_matrices(dataset):
    all_the_attention=[]
    all_the_labels=[]
    raw_attentions=[]
    klist=[9,27,45,63,81,99,117,135,153,171,189,207]
    for _,batch in enumerate(dataset):
        b_input_ids = batch[0].to(device)
        b_input_mask=attention_mask_from_input(b_input_ids)
        b_labels = batch[1].to(device)
        layer_activations = ex.get_activations(b_input_ids)
        for i in range(len(batch[0])):
            keepers=np.where(b_input_mask[i]>0)
            tmp2=keepers[0]
            raw_attention=np.zeros((12,12,len(tmp2),len(tmp2)))
            for ind_k,k in enumerate(klist):
                tmp=layer_activations[k][i].detach().numpy()
                tmp3=tmp[:,tmp2,:] # subset with the attention mask
                raw_attention[ind_k,:,:,:]=tmp3[:,:,tmp2] #subset with the attention mask
            aggregated_attention=0.5*(raw_attention+raw_attention.transpose(0,1,3,2))
            raw_attentions.append(raw_attention)
            all_the_attention.append(aggregated_attention)
            all_the_labels.append(b_labels[i])
    return(all_the_attention,all_the_labels,raw_attentions)

In [ ]:
all_the_attention_T, all_the_labels_T, raw_attentions_T = compute_attention_matrices(dl_tr)

### Some Sanity Checks
Below we plot some of the attention matrices. First the raw attention matrices, then the mean aggregated ones. We'll plot these as an example, so we can later see what kind of persistent diagrams/ images these matrices produce. 

In [ ]:
# Plot the raw attention matrices for the last training sentence:
fig, axs = plt.subplots(12,12)
fig.suptitle('Raw Attention Matrices')

for i in range(raw_attentions_T[-1].shape[0]):
    for j in range(raw_attentions_T[-1].shape[1]):
        axs[i,j].axis('off')
        axs[i,j].imshow(raw_attentions_T[-1][i,j,:,:], cmap='hot', interpolation='nearest')

In [ ]:
# Plot the mean aggregated attention matrices for the last training sentence:
fig, axs = plt.subplots(12,12)
fig.suptitle('Mean Aggregated Matrices')

for i in range(all_the_attention_T[-1].shape[0]):
    for j in range(all_the_attention_T[-1].shape[1]):
        axs[i,j].axis('off')
        axs[i,j].imshow(all_the_attention_T[-1][i,j,:,:], cmap='hot', interpolation='nearest')


In [ ]:
# Now compute the attention matrices for the validation set
# We will not be needing the raw attention matrices for the validation set
all_the_attention_V, all_the_labels_V,_ = compute_attention_matrices(dl_val)

### 3.2 Persistence Diagrams

Next we'll compute the Persistent diagrams. We'll use *gudhi* library for this. The specifications are as in the preprint:
1. The filtration value for the undirected edge $(i,j)$ is $1-f(e_{ij},e_{ji})$, where $f$ is the mean of directed edges (See Section 3.2)
2. All the vertices are born at 0; (That is: ```np.fill_diagonal(dm,0)```)
3. All the essential features are set to die at 1; (```d0[np.isinf(d0)] = 1; d1[np.isinf(d1)] = 1```)
4. The maximum dimension is 1. (```max_dimension=1; persistence_dim_max=True)```)

See Figure 3 in the preprint for illustrations.

In [ ]:
# A helper function for computing persistence diagrams from mean aggregated attention matrices
# In: 1) list_of_attention_matrices: A list of attention matrices (as returned by compute_attention_matrices)
#     2) Optional: threshold: a lowerbound for the persistent features to be computed
#     (Any feature whose persistence (i.e. d-b) is lower than the threshold will be ignored)
#     (Setting threshold to -1.0 returns all features regardless of their persistence)
# Out: Two lists of persistence diagrams, one for each homology dimension
#      These are unpacked lists, meaning that
#      the first 144 entries correspond to the first sentence, the next 144 to the next one and so on.
#      The fist 12 entries are for the Attention heads from the first layer (L=0),
#      and after that the second layer and so on all the way until 144 after which we go to the next sentece.
def persistence_diagrams_from_list_attention_matrix(list_of_attention_matrices,threshold=0.0001):
    diagramsh0=[]
    diagramsh1=[]
    for i in range(len(list_of_attention_matrices)):
        graph=list_of_attention_matrices[i] # Pick a sentence
        for j in range(graph.shape[0]): # Loop over L
            for k in range(graph.shape[1]): # Loop over H
                dm=1-graph[j,k,:,:] # The filtration value for edges is 1- f(e_i,e_j)
                np.fill_diagonal(dm,0) # Set the vertices to be born at 0
                gudhiC=gRC(distance_matrix=dm)
                simplex_tree = gudhiC.create_simplex_tree(max_dimension=1)
                diag = simplex_tree.persistence(min_persistence=threshold,persistence_dim_max=True)
                d0=simplex_tree.persistence_intervals_in_dimension(0)
                d1=simplex_tree.persistence_intervals_in_dimension(1)
                d0[np.isinf(d0)] = 1 # Set H_0 essential features to die at 1
                d1[np.isinf(d1)] = 1 # Set H_1 essential features to die at 1
                diagramsh0.append(d0)
                diagramsh1.append(d1)
    return(diagramsh0,diagramsh1)

In [ ]:
# Use the above function to compute the persistence diagrams
diagramsh0_train,diagramsh1_train=persistence_diagrams_from_list_attention_matrix(all_the_attention_T)
diagramsh0_valid,diagramsh1_valid=persistence_diagrams_from_list_attention_matrix(all_the_attention_V)

### Some Sanity Checks
Let's plot the Persistent diagrams for the attention matrices we checked earlier. Here we plot all the 144 diagrams: As such they are quite difficult to read. Because these change from run to run, you should hand pick some for closer inspection. Take a look at the matrices above, select a few that look different and look at the corresponding diagrams to get intuition how the diagrams turn out to be.

In [ ]:
# Take the last 144 persistent diagrams, corresponding to the last sentence
tmp_d0=diagramsh0_train[-144:]
tmp_d1=diagramsh1_train[-144:]

In [ ]:
fig, axs = plt.subplots(12,12)
fig.suptitle('Persistence Diagrams')
for i in range(12):
    for j in range(12):
        ind=12*i+j
        axs[i,j].axis('off')
        axs[i,j].scatter(tmp_d0[ind][:,0],tmp_d0[ind][:,1],c='blue', s=0.01)
        axs[i,j].scatter(tmp_d1[ind][:,0],tmp_d1[ind][:,1],c='red', s=0.01)
        line = mlines.Line2D([0, 1], [0, 1], color='black', linewidth=0.05)
        axs[i,j].add_line(line)

### 3.3 Persistent Images

We can now compute the Persistent Images. The parameters are adapted from the preprint.

In [ ]:
pers0=gPI(bandwidth=0.1, weight=lambda x: 1,im_range=[0,0.01,0,1], resolution=[5,50])
PI0t=pers0.fit_transform(diagramsh0_train)
PI0v=pers0.fit_transform(diagramsh0_valid)
pers1=gPI(bandwidth=0.1, weight=lambda x: 1,im_range=[0,1,0.99,1], resolution=[50,5])
PI1t=pers1.fit_transform(diagramsh1_train)
PI1v=pers1.fit_transform(diagramsh1_valid)

The data from the persistent Image computations is a list, which is not fantastic for bookkeeping. We'll convert the data into proper images of size 5x50 with 288 channels:

In [ ]:
# A helper funcion for formatting the persistence images as proper images
# In: H0 (An array of persistence images for H0 as returned by fit_transform
#         of size (n x 144, 250)), where n is the sample size (number of sentences)
#     H1 (An array of persistence images for H1 as returned by fit_transform
#         of size (n x 144, 250))
# Out: an array of size (n,288,50,5)
#      the second dimension is the attention heads concatenated with homology dimension
#      This is the same setup as in the preprint, and allows for a direct application of
#      a convolutional neural network
#      (note that the simple model in this notebook doesn't make use of this structure)

def reshape_persistence_images(H0,H1):
    H0_reshaped=H0.reshape(-1,144,250)
    H0_reshaped=H0_reshaped.reshape(-1,144,50,5)
    H0_reshaped=H0_reshaped.transpose(0,1,3,2)
    H1_reshaped=H1.reshape(-1,144,250)
    H1_reshaped=H1_reshaped.reshape(-1,144,50,5)
    H1_reshaped=H1_reshaped.transpose(0,1,3,2)
    stack=np.concatenate([H0_reshaped,H1_reshaped],axis=1)
    return(stack)

In [ ]:
# For the sake of transparency (and visualization), we'll reshape our Persistent Images:
images_train=reshape_persistence_images(PI0t,PI1t)
images_val=reshape_persistence_images(PI0v,PI1v)

### Some Sanity Checks
Let's have a look at the persistent images for the matrices we saw earlier. Again, you should investigate a specific image to get a better idea of what is going on.

In [ ]:
# Again we pick the last sentence from the train dataset
tmp_im0=images_train[-1,:144,:,:] #H0
tmp_im1=images_train[-1,144:,:,:] #H1
tmp_im0=tmp_im0.transpose(0,2,1)
tmp_im1=tmp_im1.transpose(0,2,1)

In [ ]:
fig, axs = plt.subplots(12,12)
fig.suptitle('Images H0')

for k in range(tmp_im0.shape[0]):
    j=math.floor(k/12)
    i=k%12
    axs[i,j].axis('off')
    axs[i,j].imshow(tmp_im0[k,:,:], cmap='hot', interpolation='nearest')

In [ ]:
fig, axs = plt.subplots(12,12)
fig.suptitle('Images H1')

for k in range(tmp_im1.shape[0]):
    j=math.floor(k/12)
    i=k%12
    axs[i,j].axis('off')
    axs[i,j].imshow(tmp_im1[k,:,:], cmap='hot', interpolation='nearest')

### A hand-picked example
Here we look at a specific example coming from the 4th attention head (H=3) on the 1st (L=0) Multi-Head attention layer. We'll plot the Raw attention matrix, the mean aggregated one as well as the persistent diagram and persistent images.

In [ ]:
L=0
H=3
k=12*L+H

plt.imshow(raw_attentions_T[-1][L,H,:,:], cmap='hot', interpolation='nearest')
plt.title("Raw Attention Matrix")
plt.show()

plt.imshow(all_the_attention_T[-1][L,H,:,:], cmap='hot', interpolation='nearest')
plt.title("The Mean")
plt.show()

In [ ]:
# The persistent diagram from the matrix above:
h0plot=plt.scatter(tmp_d0[k][:,0],tmp_d0[k][:,1],c='blue', s=3)
h1plot=plt.scatter(tmp_d1[k][:,0],tmp_d1[k][:,1],c='red', s=3)
plt.plot([0, 1], c='black')
plt.legend((h0plot,h1plot),
           ('H_0', 'H_1'),
           scatterpoints=3,
           loc='lower right',
           ncol=2,
           fontsize=12)
plt.title("Resulting Persistent Diagram")
plt.show()

fig, axs = plt.subplots(1,2)
fig.suptitle("Persistent Images")
axs[0].imshow(tmp_im0[k,:,:], cmap='hot', interpolation='nearest')
axs[0].title.set_text('H_0')
axs[1].imshow(tmp_im1[k,:,:], cmap='hot', interpolation='nearest')
axs[1].title.set_text('H_1')

## 4. Training a topological model
We'll train a super simple model based on our topological summaries. This follows the usual *giotto-deep* recipe:

In [ ]:
dl_builder2 = DataLoaderBuilder((FromArray(images_train, train_labels), FromArray(images_val, validation_labels)))
dl_tr2, dl_val2, _ = dl_builder2.build(({"batch_size": 8}, {"batch_size": 8}))

Our model is just a one layer linear model mapping from $\mathbb{R}^{(288x50x5)}$ to $\mathbb{R}^2$.

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear=nn.Linear(288*50*5,2)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = x.type(torch.FloatTensor)
        x = self.linear(x)
        return x


net = Net()



criterion = nn.CrossEntropyLoss()
optimizer = Adam(net.parameters(), lr=0.000067)

In [ ]:
pipe2 = Trainer(net, (dl_tr2, dl_val2), criterion, writer)

In [ ]:
pipe2.train(
    Adam,
    20,
    False,
    lr_scheduler=ExponentialLR,
    scheduler_params={"gamma": 0.9},
    profiling=False,
    store_grad_layer_hist=True,
    writer_tag="line",
)

## 5. Pruning the Model

Compared to the complicated BERT model, the topological model we just defined is quite transparent in that we can easily define importance metrics for the attention heads.

One way to do this is to do what was done in the preprint (See Section 6 therein). This is exactly what we do here.

We compute the gradient of the output logits with respect to the input image, that is, a stack of persistent image of pixels arranged in an array of size $(288,50,5)$. The gradient is of the same size $(288,50,5)$ and it measures how much the response changes as we change the values of the pixels. If the grad is low in absolute value, changing it would not affect the final classification by much. This the said pixel does not help much in the classification task, the prediction is would be similar regardless of what the actual value was.

As we are only interested how the response changes with respect to each attention heads, we need to aggregate the gradients over a) The homology dimension $H_0$ and $H_1$ b) the pixel values $(50,5)$. We can do this by summing over the values (this is equivalent to taking the mean, as each head is summed over $nx2x50x5$ values, where $n$ is the number of sentences considered.

We do this as follows:
1. compute the gradient of the output logits with respect to the input image for $n$ many sentences
2. For each attention head $i$, $i=1,2,\ldots, 144$, compute the sum over the $nx2x50x5$ many values


First we need to format our data to compute the gradient. This is the content of the next code chunk:

In [ ]:
tmp1=torch.FloatTensor(images_train)
train_labels=torch.FloatTensor(train_labels)
validation_labels=torch.FloatTensor(validation_labels)
train_labels = train_labels.type(torch.LongTensor)
validation_labels = validation_labels.type(torch.LongTensor)
batch_size=3
tmp2=torch.FloatTensor(images_val)
tmp2.requires_grad=True
ds1 = TensorDataset(tmp1, train_labels)
ds2 = TensorDataset(tmp2, validation_labels)
trainloader = torch.utils.data.DataLoader(ds1, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(ds2, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

In [ ]:
# Step 1. Here we compute the gradients for the first 500 images in our training dataset.
imageset=trainloader.dataset[0:500][0]
target_label=trainloader.dataset[0:500][1]
a = torch.autograd.Variable(imageset)
a.requires_grad=True
outputs=net(a)
target_label
loss = criterion(outputs,target_label)
a_grad = torch.autograd.grad(loss, a, retain_graph=True)

In [ ]:
# Step 2. Take the absolute value of the gradients, and sum them over axes
# 0 (the samples) and
# (2,3) (The actual persistent image pixels)
tmp=np.sum(abs(a_grad[0].detach().cpu().numpy()), axis=(0,2,3))
# Average over hom.dim (H0+H1) (first 144 are H0, the rest are H1)
tmp2=tmp[0:144]+tmp[144:] 

# Unpack the scores over 144 attention heads to 12 by 12 matrix, corresponding to H and L 
tmp3=tmp2.reshape(12,12)
# Plot the values
plt.imshow(tmp3, cmap='hot', interpolation='nearest')
plt.ylabel('L (depth)')
plt.xlabel('H (attention head)')
plt.title('Importance Scores for Attention heads')
plt.colorbar()
plt.show()

We can now select the most impactful however many heads we want. In the code below we take the top 100. We'll stick to the simple model we used earlier, so you only need to change
```
heads_to_keep
```
in the code chunk below to select however many heads you want.

In [ ]:
heads_to_keep=100
pruned_indices=np.argsort(tmp2)[-heads_to_keep:]

In [ ]:
# The pruned indices refer to the attention heads. As we stacked the homology H0 and H1,
# the indices for attention head i are
#                           i for H0 and
#                           (i+144) for H1)
pt=np.concatenate((pruned_indices,pruned_indices+144),axis=0)
images_train=images_train[:,pt,:,:]
images_val=images_val[:,pt,:,:]
images_train= images_train.astype(np.float32)
images_val= images_val.astype(np.float32)

In [ ]:
dl_builder2 = DataLoaderBuilder((FromArray(images_train, train_labels), FromArray(images_val, validation_labels)))
dl_tr2, dl_val2, _ = dl_builder2.build(({"batch_size": 8}, {"batch_size": 8}))

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear=nn.Linear(2*heads_to_keep*50*5,2)

    def forward(self, x):
        x = torch.flatten(x, 1) 
        x = self.linear(x)
        return x


net = Net()

criterion = nn.CrossEntropyLoss()
optimizer = Adam(net.parameters(), lr=0.000067)

In [ ]:
pipe2 = Trainer(net, (dl_tr2, dl_val2), criterion, writer)
# train the model with learning rate scheduler
pipe2.train(
    Adam,
    20,
    False,
    lr_scheduler=ExponentialLR,
    scheduler_params={"gamma": 0.9},
    profiling=False,
    store_grad_layer_hist=True,
    writer_tag="line",
)

## Conclusion

In this tutorial, we used *giotto-deep* to
1. Fit an off the shelf Huggingface model;
2. Fine tune it;
3. Extract its inner workings to;
4. construct a pruned model.

Depending on the choices made along the way, the pruned model may be anything from strong to terrible. To increase performance, you may

1. Add more data (remember this notebook is heavily subsampled);
2. Spend more time fine-tuning the BERT; 
3. Fit more complicated models;
4. Optimize the parameters.
